# Pre-Equilibrium Maximazation of Yield #

In [1]:
# make sure jupyter path is correct for loading local moudules
import sys
# path to steric_simulator module relative to notebook
sys.path.append("../../")
import copy

In [2]:
from steric_free_simulator import ReactionNetwork, VectorizedRxnNet, VecSim, Optimizer, EquilibriumSolver

EnergyExplorer Module is not available. Check Rosetta installation. <ipykernel.iostream.OutStream object at 0x7f1e32dd2da0>


We'll start with the AP2 complex that we've worked with before. Pairwise $\Delta Gs$ were derived from the PDB structures via Rossetta

In [3]:
base_input = '../input_files/ap2.pwr'
rn = ReactionNetwork(base_input, one_step=True)
rn.resolve_tree()
vec_rn = VectorizedRxnNet(rn, dev='cpu')

['A']
['M']
['B']
['S']
-----
{'A'}
{'A'}
set()
-----
{'A'}
{'M'}
{'A'}
False
Allowed edges: 
('A', 'B')
Allowed edges: 
('A', 'M')
Connected Nodes:  ['A', 'M']
Connected Edges:  [('A', 'M')]
Allowed edges: 
('A', 'S')
Allowed edges: 
('B', 'M')
Allowed edges: 
('M', 'S')
New node added--1
['A', 'M']
{('A', 'B'): [None, None, 1, tensor([1650.], dtype=torch.float64)], ('A', 'M'): [None, None, 1, tensor([-1300.], dtype=torch.float64)], ('A', 'S'): [None, None, 1, tensor([1660.], dtype=torch.float64)], ('B', 'M'): [None, None, 1, tensor([2100.], dtype=torch.float64)], ('M', 'S'): [None, None, 1, tensor([-4050.], dtype=torch.float64)]}
New node added--2
[0, 1, 2, 3, 4]
-----
{'A'}
{'B'}
{'A'}
False
Allowed edges: 
('A', 'B')
Connected Nodes:  ['A', 'B']
Connected Edges:  [('A', 'B')]
Allowed edges: 
('A', 'M')
Allowed edges: 
('A', 'S')
Allowed edges: 
('B', 'M')
Allowed edges: 
('M', 'S')
New node added--1
['A', 'B']
{('A', 'B'): [None, None, 1, tensor([1650.], dtype=torch.float64)], ('A'

In [ ]:
#Do modifications here

#Changing initial concentrations

## The Equilibrium Solution ##
First we will find the equilibrium solution for this system.

In [4]:
vec_rn.update_reaction_net(rn)
poly_system = EquilibriumSolver(rn)
solution = poly_system.solve()
print(solution)
print("Equilibrium expected yield: ", 100 * solution[-1] / min(vec_rn.initial_copies[:vec_rn.num_monomers]), '%')

Matrix([[1.047897], [0.03076379], [1.920964], [1.587516], [0.05714356], [0.9734105], [0.8009079], [0.02344015], [0.2905829], [0.02105455], [0.2598622], [0.7439778], [0.2214066], [0.09574625]])
Equilibrium expected yield:  9.57462501525879 %


In [5]:
uid_dict = {}
sys.path.append("../")
import numpy as np
from reaction_network import gtostr
from torch import DoubleTensor as Tensor

def get_max_edge(n):
    """
    Calculates the max rate (k_on) for a given node
    To find out the maximum flow path to the final complex starting from the current node.
    
    Can also calculate the total rate of consumption of a node by summing up all rates. 
    Can tell which component is used quickly.
    """
    try:
        edges = rn.network.out_edges(n)
        #Loop over all edges
        #Get attributes
        if len(edges)==0:
            return(False)
        kon_max = -1
        next_node = -1
        
        kon_sum = 0
        for edge in edges:
            data = rn.network.get_edge_data(edge[0],edge[1])
            #print(data)
            #Get uid
            uid = data['uid']
            #Get updated kon
            temp_kon = vec_rn.kon[uid]
            kon_sum+=temp_kon
            
#             #Calculate k_off also
#             std_c = Tensor([1.])
#             l_kon = torch.log(temp_kon)
#             l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (self._R * self._T)) + l_kon + torch.log(std_c)
            if temp_kon > kon_max:
                kon_max = temp_kon
                next_node=edge[1]
        return(kon_max,next_node,kon_sum)
    except Exception as err:
        raise(err)

pathway = []
kon_sumarray = []
total_con_rate = {}
for n in rn.network.nodes():
    
    n_str = gtostr(rn.network.nodes[n]['struct']) 
    
    paths = [n_str]
    kon_sum = 0
    temp_node = n
    max_edge = True
    consumption_rate = 0
    if n < len(rn.network.nodes()):#num_monomers:
#         print("Current node: ")
#         print(n_str)
        while max_edge:
            max_edge = get_max_edge(temp_node)
            if max_edge:
                total_con_rate[gtostr(rn.network.nodes[temp_node]['struct'])] = max_edge[2]
                temp_node = max_edge[1]
                kon_sum += max_edge[0].item()
                
#                 print("Next node: ")
#                 print(temp_node)

                paths.append(gtostr(rn.network.nodes[temp_node]['struct']))
            else:
                break
        pathway.append(paths)
        kon_sumarray.append(kon_sum)
        paths=[]

print(pathway)
print(kon_sumarray)
#print(total_con_rate)

[['A', 'AM', 'ABM', 'ABMS'], ['M', 'AM', 'ABM', 'ABMS'], ['B', 'AB', 'ABM', 'ABMS'], ['S', 'AS', 'AMS', 'ABMS'], ['AM', 'ABM', 'ABMS'], ['AB', 'ABM', 'ABMS'], ['AS', 'AMS', 'ABMS'], ['BM', 'BMS', 'ABMS'], ['MS', 'BMS', 'ABMS'], ['ABM', 'ABMS'], ['AMS', 'ABMS'], ['ABS', 'ABMS'], ['BMS', 'ABMS'], ['ABMS']]
[3.0, 3.0, 3.0, 3.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 0]


In [6]:
for k,v in sorted(total_con_rate.items(),key=lambda x : x[1]):
    print(k," : ", v.item())

ABM  :  1.0
AMS  :  1.0
BMS  :  1.0
ABS  :  1.0
AM  :  2.0
AB  :  3.0
AS  :  3.0
BM  :  3.0
MS  :  3.0
A  :  6.0
M  :  6.0
B  :  6.0
S  :  6.0


## Using the optimizer with a 1 second simulation runtime ##

In [7]:
vec_rn.reset(reset_params=True)
optim = Optimizer(reaction_network=vec_rn,
                  sim_runtime=1,
                  optim_iterations=5000,
                  learning_rate=.001,
                  device='cpu')
optim.rn.update_reaction_net(rn)
optim.optimize()

Using CPU
Reaction Parameters before optimization: 
[Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1.], dtype=torch.float64, requires_grad=True)]
Using CPU
yield on sim iteration 0 was 9.36%
current params: tensor([0.9990, 0.9990, 0.9990, 0.9990, 1.0010, 0.9990, 1.0010, 0.9990, 0.9990,
        0.9990, 1.0010, 1.0010, 0.9990, 1.0010, 1.0010, 0.9990, 1.0010, 0.9990,
        0.9990, 1.0010, 0.9990], dtype=torch.float64)
Using CPU
yield on sim iteration 1 was 9.36%
current params: tensor([0.9980, 0.9980, 0.9980, 0.9980, 1.0020, 0.9980, 1.0020, 0.9980, 0.9980,
        0.9980, 1.0020, 1.0020, 0.9980, 1.0020, 1.0020, 0.9980, 1.0020, 0.9980,
        0.9980, 1.0020, 0.9980], dtype=torch.float64)
Using CPU
yield on sim iteration 2 was 9.36%
current params: tensor([0.9970, 0.9970, 0.9970, 0.9970, 1.0030, 0.9970, 1.0030, 0.9970, 0.9970,
        0.9970, 1.0030, 1.0030, 0.9970, 1.0030, 1.0030, 0.9970, 1.0030, 0.9970,
        0.9

current params: tensor([0.9604, 0.9555, 0.9551, 0.9546, 1.0435, 0.9561, 1.0448, 0.9560, 0.9556,
        0.9551, 1.0454, 1.0446, 0.9552, 1.0428, 1.0445, 0.9545, 1.0446, 0.9570,
        0.9551, 1.0444, 0.9547], dtype=torch.float64)
Using CPU
yield on sim iteration 45 was 9.41%
current params: tensor([0.9598, 0.9545, 0.9541, 0.9537, 1.0444, 0.9552, 1.0458, 0.9551, 0.9547,
        0.9541, 1.0465, 1.0455, 0.9543, 1.0437, 1.0454, 0.9535, 1.0456, 0.9561,
        0.9542, 1.0454, 0.9537], dtype=torch.float64)
Using CPU
yield on sim iteration 46 was 9.41%
current params: tensor([0.9592, 0.9536, 0.9531, 0.9527, 1.0453, 0.9543, 1.0467, 0.9542, 0.9538,
        0.9531, 1.0475, 1.0465, 0.9533, 1.0445, 1.0464, 0.9525, 1.0465, 0.9553,
        0.9532, 1.0463, 0.9527], dtype=torch.float64)
Using CPU
yield on sim iteration 47 was 9.41%
current params: tensor([0.9587, 0.9527, 0.9522, 0.9517, 1.0461, 0.9534, 1.0477, 0.9533, 0.9528,
        0.9522, 1.0485, 1.0474, 0.9524, 1.0454, 1.0473, 0.9515, 1.0475, 0.95

Using CPU
yield on sim iteration 79 was 9.44%
current params: tensor([0.9572, 0.9241, 0.9224, 0.9208, 1.0711, 0.9285, 1.0770, 0.9274, 0.9247,
        0.9218, 1.0808, 1.0766, 0.9230, 1.0687, 1.0759, 0.9199, 1.0761, 0.9329,
        0.9206, 1.0753, 0.9203], dtype=torch.float64)
Using CPU
yield on sim iteration 80 was 9.44%
current params: tensor([0.9578, 0.9232, 0.9214, 0.9198, 1.0719, 0.9278, 1.0779, 0.9266, 0.9239,
        0.9209, 1.0819, 1.0775, 0.9221, 1.0694, 1.0768, 0.9189, 1.0770, 0.9324,
        0.9196, 1.0762, 0.9192], dtype=torch.float64)
Using CPU
yield on sim iteration 81 was 9.44%
current params: tensor([0.9585, 0.9223, 0.9205, 0.9188, 1.0726, 0.9271, 1.0788, 0.9258, 0.9230,
        0.9199, 1.0829, 1.0785, 0.9212, 1.0700, 1.0777, 0.9178, 1.0779, 0.9319,
        0.9186, 1.0771, 0.9182], dtype=torch.float64)
Using CPU
yield on sim iteration 82 was 9.44%
current params: tensor([0.9592, 0.9214, 0.9195, 0.9178, 1.0733, 0.9264, 1.0798, 0.9251, 0.9221,
        0.9189, 1.0840, 1.0794

Using CPU
yield on sim iteration 111 was 9.48%
current params: tensor([0.9949, 0.8964, 0.8920, 0.8876, 1.0914, 0.9089, 1.1067, 0.9038, 0.8980,
        0.8909, 1.1154, 1.1057, 0.8936, 1.0850, 1.1042, 0.8861, 1.1048, 0.9238,
        0.8890, 1.1030, 0.8871], dtype=torch.float64)
Using CPU
yield on sim iteration 112 was 9.48%
current params: tensor([0.9965, 0.8956, 0.8911, 0.8866, 1.0920, 0.9084, 1.1076, 0.9031, 0.8972,
        0.8900, 1.1165, 1.1066, 0.8927, 1.0854, 1.1051, 0.8850, 1.1057, 0.9238,
        0.8880, 1.1038, 0.8861], dtype=torch.float64)
Using CPU
yield on sim iteration 113 was 9.49%
current params: tensor([0.9981, 0.8948, 0.8902, 0.8855, 1.0925, 0.9080, 1.1085, 0.9025, 0.8965,
        0.8890, 1.1176, 1.1075, 0.8918, 1.0858, 1.1059, 0.8840, 1.1065, 0.9238,
        0.8870, 1.1047, 0.8850], dtype=torch.float64)
Using CPU
yield on sim iteration 114 was 9.49%
current params: tensor([0.9997, 0.8940, 0.8893, 0.8845, 1.0930, 0.9075, 1.1094, 0.9018, 0.8957,
        0.8881, 1.1187, 1.

yield on sim iteration 143 was 9.52%
current params: tensor([1.0498, 0.8726, 0.8638, 0.8550, 1.1039, 0.8994, 1.1342, 0.8863, 0.8758,
        0.8613, 1.1510, 1.1329, 0.8668, 1.0918, 1.1302, 0.8521, 1.1311, 0.9340,
        0.8555, 1.1281, 0.8534], dtype=torch.float64)
Using CPU
yield on sim iteration 144 was 9.52%
current params: tensor([1.0516, 0.8719, 0.8630, 0.8540, 1.1041, 0.8992, 1.1350, 0.8859, 0.8751,
        0.8604, 1.1521, 1.1337, 0.8660, 1.0918, 1.1310, 0.8510, 1.1319, 0.9346,
        0.8545, 1.1288, 0.8523], dtype=torch.float64)
Using CPU
yield on sim iteration 145 was 9.53%
current params: tensor([1.0534, 0.8712, 0.8621, 0.8530, 1.1044, 0.8991, 1.1358, 0.8854, 0.8745,
        0.8595, 1.1533, 1.1346, 0.8653, 1.0919, 1.1318, 0.8499, 1.1326, 0.9353,
        0.8534, 1.1296, 0.8513], dtype=torch.float64)
Using CPU
yield on sim iteration 146 was 9.53%
current params: tensor([1.0552, 0.8706, 0.8613, 0.8520, 1.1046, 0.8990, 1.1367, 0.8850, 0.8739,
        0.8586, 1.1544, 1.1354, 0.86

...

yield on sim iteration 4863 was 21.4%
current params: tensor([4.3050, 4.2871, 3.9914, 0.0177, 0.0608, 4.6959, 4.0512, 4.4549, 4.5003,
        0.0188, 4.7855, 3.2645, 4.3593, 0.0757, 3.1478, 0.0137, 0.0544, 0.0290,
        0.0139, 3.7750, 0.0132], dtype=torch.float64)
Using CPU
yield on sim iteration 4864 was 21.4%
current params: tensor([4.3056, 4.2880, 3.9921, 0.0177, 0.0607, 4.6966, 4.0517, 4.4558, 4.5012,
        0.0188, 4.7863, 3.2648, 4.3601, 0.0757, 3.1480, 0.0136, 0.0553, 0.0290,
        0.0142, 3.7754, 0.0131], dtype=torch.float64)
Using CPU
yield on sim iteration 4865 was 21.4%
current params: tensor([4.3062, 4.2888, 3.9929, 0.0177, 0.0606, 4.6974, 4.0522, 4.4567, 4.5020,
        0.0188, 4.7871, 3.2650, 4.3609, 0.0757, 3.1482, 0.0135, 0.0561, 0.0289,
        0.0145, 3.7758, 0.0130], dtype=torch.float64)
Using CPU
yield on sim iteration 4866 was 21.4%
current params: tensor([4.3068, 4.2896, 3.9936, 0.0176, 0.0605, 4.6981, 4.0527, 4.4575, 4.5029,
        0.0188, 4.7879, 3.2653, 

Using CPU
yield on sim iteration 4897 was 21.5%
current params: tensor([4.3246, 4.3152, 4.0168, 0.0154, 0.0599, 4.7206, 4.0682, 4.4844, 4.5293,
        0.0183, 4.8125, 3.2741, 4.3878, 0.0754, 3.1562, 0.0113, 0.0617, 0.0280,
        0.0143, 3.7891, 0.0131], dtype=torch.float64)
Using CPU
yield on sim iteration 4898 was 21.4%
current params: tensor([4.3251, 4.3160, 4.0175, 0.0153, 0.0597, 4.7213, 4.0687, 4.4852, 4.5302,
        0.0183, 4.8133, 3.2744, 4.3886, 0.0754, 3.1565, 0.0118, 0.0617, 0.0279,
        0.0142, 3.7896, 0.0134], dtype=torch.float64)
Using CPU
yield on sim iteration 4899 was 21.4%
current params: tensor([4.3257, 4.3168, 4.0182, 0.0152, 0.0596, 4.7221, 4.0692, 4.4861, 4.5310,
        0.0183, 4.8141, 3.2746, 4.3894, 0.0753, 3.1567, 0.0123, 0.0616, 0.0279,
        0.0141, 3.7900, 0.0137], dtype=torch.float64)
Using CPU
yield on sim iteration 4900 was 21.4%
current params: tensor([4.3263, 4.3176, 4.0190, 0.0152, 0.0594, 4.7228, 4.0697, 4.4869, 4.5318,
        0.0183, 4.8148

yield on sim iteration 4930 was 21.5%
current params: tensor([4.3431, 4.3425, 4.0417, 0.0127, 0.0607, 4.7442, 4.0842, 4.5132, 4.5577,
        0.0178, 4.8395, 3.2832, 4.4156, 0.0751, 3.1641, 0.0138, 0.0598, 0.0269,
        0.0107, 3.8023, 0.0135], dtype=torch.float64)
Using CPU
yield on sim iteration 4931 was 21.5%
current params: tensor([4.3436, 4.3433, 4.0424, 0.0126, 0.0608, 4.7449, 4.0847, 4.5141, 4.5586,
        0.0178, 4.8403, 3.2835, 4.4165, 0.0751, 3.1644, 0.0137, 0.0597, 0.0269,
        0.0106, 3.8027, 0.0134], dtype=torch.float64)
Using CPU
yield on sim iteration 4932 was 21.5%
current params: tensor([4.3442, 4.3442, 4.0431, 0.0125, 0.0609, 4.7456, 4.0851, 4.5150, 4.5594,
        0.0178, 4.8411, 3.2837, 4.4173, 0.0751, 3.1646, 0.0136, 0.0596, 0.0268,
        0.0105, 3.8030, 0.0133], dtype=torch.float64)
Using CPU
yield on sim iteration 4933 was 21.5%
current params: tensor([4.3447, 4.3450, 4.0439, 0.0124, 0.0610, 4.7463, 4.0856, 4.5158, 4.5603,
        0.0178, 4.8419, 3.2840, 

current params: tensor([4.3604, 4.3678, 4.0650, 0.0099, 0.0609, 4.7662, 4.0993, 4.5400, 4.5841,
        0.0173, 4.8646, 3.2919, 4.4417, 0.0748, 3.1717, 0.0101, 0.0575, 0.0259,
        0.0155, 3.8147, 0.0113], dtype=torch.float64)
Using CPU
yield on sim iteration 4963 was 21.5%
current params: tensor([4.3609, 4.3686, 4.0657, 0.0109, 0.0609, 4.7669, 4.0997, 4.5408, 4.5849,
        0.0173, 4.8653, 3.2922, 4.4425, 0.0748, 3.1719, 0.0100, 0.0574, 0.0258,
        0.0154, 3.8151, 0.0119], dtype=torch.float64)
Using CPU
yield on sim iteration 4964 was 21.5%
current params: tensor([4.3615, 4.3694, 4.0664, 0.0118, 0.0609, 4.7676, 4.1002, 4.5417, 4.5857,
        0.0173, 4.8661, 3.2924, 4.4432, 0.0748, 3.1722, 0.0099, 0.0574, 0.0258,
        0.0154, 3.8155, 0.0123], dtype=torch.float64)
Using CPU
yield on sim iteration 4965 was 21.5%
current params: tensor([4.3620, 4.3701, 4.0671, 0.0126, 0.0608, 4.7682, 4.1007, 4.5425, 4.5865,
        0.0173, 4.8668, 3.2927, 4.4440, 0.0748, 3.1724, 0.0106, 0.0573

current params: tensor([4.3789, 4.3950, 4.0898, 0.0177, 0.0605, 4.7899, 4.1152, 4.5687, 4.6123,
        0.0168, 4.8912, 3.3009, 4.4695, 0.0745, 3.1795, 0.0142, 0.0552, 0.0248,
        0.0124, 3.8279, 0.0139], dtype=torch.float64)
Using CPU
yield on sim iteration 4996 was 21.4%
current params: tensor([4.3795, 4.3959, 4.0906, 0.0176, 0.0603, 4.7906, 4.1157, 4.5696, 4.6132,
        0.0168, 4.8920, 3.3012, 4.4703, 0.0745, 3.1798, 0.0141, 0.0551, 0.0247,
        0.0123, 3.8283, 0.0138], dtype=torch.float64)
Using CPU
yield on sim iteration 4997 was 21.4%
current params: tensor([4.3801, 4.3968, 4.0914, 0.0176, 0.0602, 4.7913, 4.1162, 4.5705, 4.6141,
        0.0168, 4.8929, 3.3015, 4.4712, 0.0745, 3.1800, 0.0140, 0.0550, 0.0247,
        0.0122, 3.8287, 0.0137], dtype=torch.float64)
Using CPU
yield on sim iteration 4998 was 21.4%
current params: tensor([4.3807, 4.3976, 4.0921, 0.0175, 0.0601, 4.7921, 4.1167, 4.5714, 4.6150,
        0.0168, 4.8937, 3.3017, 4.4721, 0.0745, 3.1803, 0.0139, 0.0550

In [8]:
uid_dict = {}
sys.path.append("../")
import numpy as np
from reaction_network import gtostr
for n in rn.network.nodes():
    #print(n)
    #print(rn.network.nodes()[n])
    for k,v in rn.network[n].items():
        uid = v['uid']
        r1 = set(gtostr(rn.network.nodes[n]['struct']))
        p = set(gtostr(rn.network.nodes[k]['struct']))
        r2 = p-r1
        reactants = (r1,r2)
        uid_val = {'reactants':reactants,'kon':v['k_on'],'score':v['rxn_score'],'koff':v['k_off']}
        if uid not in uid_dict.keys():
            uid_dict[uid] = uid_val
    print(gtostr(rn.network.nodes[n]['struct']))
    #for r_set in rn.get_reactant_sets(n):
    #    print(tuple(r_set))
    #print(rn.network[n]['struct'])
ind_sort = np.argsort(vec_rn.kon.detach().numpy())
for i in ind_sort:
    print(vec_rn.kon[i])
    print(uid_dict[i])

A
M
B
S
AM
AB
AS
BM
MS
ABM
AMS
ABS
BMS
ABMS
tensor(0.0121, dtype=torch.float64, grad_fn=<SelectBackward>)
{'reactants': ({'A'}, {'M', 'S'}), 'kon': 1.0, 'score': tensor([360.], dtype=torch.float64), 'koff': 1.1717784441707233}
tensor(0.0136, dtype=torch.float64, grad_fn=<SelectBackward>)
{'reactants': ({'A'}, {'M', 'S', 'B'}), 'kon': 1.0, 'score': tensor([2010.], dtype=torch.float64), 'koff': 2.4231896220651605}
tensor(0.0139, dtype=torch.float64, grad_fn=<SelectBackward>)
{'reactants': ({'A', 'B'}, {'M', 'S'}), 'kon': 1.0, 'score': tensor([2460.], dtype=torch.float64), 'koff': 2.9542298433333407}
tensor(0.0168, dtype=torch.float64, grad_fn=<SelectBackward>)
{'reactants': ({'B'}, {'M', 'S'}), 'kon': 1.0, 'score': tensor([2100.], dtype=torch.float64), 'koff': 2.521150528096693}
tensor(0.0175, dtype=torch.float64, grad_fn=<SelectBackward>)
{'reactants': ({'M'}, {'B'}), 'kon': 1.0, 'score': tensor([2100.], dtype=torch.float64), 'koff': 2.521150528096693}
tensor(0.0247, dtype=torch.float64

In [9]:
uid_dict = {}
sys.path.append("../")
import numpy as np
from reaction_network import gtostr
from torch import DoubleTensor as Tensor

def get_max_edge(n):
    """
    Calculates the max rate (k_on) for a given node
    To find out the maximum flow path to the final complex starting from the current node.
    
    Can also calculate the total rate of consumption of a node by summing up all rates. 
    Can tell which component is used quickly.
    """
    try:
        edges = rn.network.out_edges(n)
        #Loop over all edges
        #Get attributes
        if len(edges)==0:
            return(False)
        kon_max = -1
        next_node = -1
        
        kon_sum = 0
        for edge in edges:
            data = rn.network.get_edge_data(edge[0],edge[1])
            #print(data)
            #Get uid
            uid = data['uid']
            #Get updated kon
            temp_kon = vec_rn.kon[uid]
            kon_sum+=temp_kon
            
#             #Calculate k_off also
#             std_c = Tensor([1.])
#             l_kon = torch.log(temp_kon)
#             l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (self._R * self._T)) + l_kon + torch.log(std_c)
            if temp_kon > kon_max:
                kon_max = temp_kon
                next_node=edge[1]
        return(kon_max,next_node,kon_sum)
    except Exception as err:
        raise(err)

pathway = []
kon_sumarray = []
total_con_rate = {}
for n in rn.network.nodes():
    
    n_str = gtostr(rn.network.nodes[n]['struct']) 
    
    paths = [n_str]
    kon_sum = 0
    temp_node = n
    max_edge = True
    consumption_rate = 0
    if n < len(rn.network.nodes()):#num_monomers:
#         print("Current node: ")
#         print(n_str)
        while max_edge:
            max_edge = get_max_edge(temp_node)
            if max_edge:
                total_con_rate[gtostr(rn.network.nodes[temp_node]['struct'])] = max_edge[2]
                temp_node = max_edge[1]
                kon_sum += max_edge[0].item()
                
#                 print("Next node: ")
#                 print(temp_node)

                paths.append(gtostr(rn.network.nodes[temp_node]['struct']))
            else:
                break
        pathway.append(paths)
        kon_sumarray.append(kon_sum)
        paths=[]

print(pathway)
print(kon_sumarray)
#print(total_con_rate)

[['A', 'AB', 'ABM', 'ABMS'], ['M', 'ABM', 'ABMS'], ['B', 'ABMS'], ['S', 'ABS', 'ABMS'], ['AM', 'ABM', 'ABMS'], ['AB', 'ABM', 'ABMS'], ['AS', 'ABS', 'ABMS'], ['BM', 'BMS', 'ABMS'], ['MS', 'BMS', 'ABMS'], ['ABM', 'ABMS'], ['AMS', 'ABMS'], ['ABS', 'ABMS'], ['BMS', 'ABMS'], ['ABMS']]
[12.369953781777424, 7.972342706474297, 4.893721470795163, 8.30119325211303, 7.751680525316628, 7.972342706474297, 8.444124072758427, 0.08815349675307924, 0.030412966790277902, 3.180253210483606, 4.893721470795163, 3.8291386613414304, 0.013644950302059242, 0]


In [10]:
for k,v in sorted(total_con_rate.items(),key=lambda x : x[1]):
    print(k," : ", v.item())

BMS  :  0.013644950302059242
MS  :  0.04275173656798596
BM  :  0.15415046286292866
ABM  :  3.180253210483606
ABS  :  3.8291386613414304
AMS  :  4.893721470795163
AM  :  7.873176857160429
AS  :  8.786626136273947
AB  :  9.278024099731082
A  :  12.920839797789494
S  :  15.180783855402746
M  :  17.196151145837835
B  :  18.512013054379747


Let's first visualize some of the data.

**Without any optimization**


In [11]:
optim.plot_observable(0)


TypeError: plot_observable() missing 1 required positional argument: 'nodes_list'

**After 750 optimization iterations**


In [ ]:
optim.plot_observable(-1)


In [ ]:
optim.plot_yield()

It seems like we've found a stable solution that produces greater yield than equilibrium. This should be thermodynamically
impossible. Let's try to find an explanation. We'll run simulations using the learned optimal parameters at a few different
timescales.

In [ ]:
from matplotlib import pyplot as plt
fig, ax = plt.subplots(1, 3)
optim_rn = optim.rn
for i, runtime in enumerate([1, 8, 64]):
    optim_rn.reset()
    sim = VecSim(optim_rn, runtime, device='cpu')
    y = sim.simulate()
    sim.plot_observable(ax=ax[i])
    ax[i].set_title("runtime: " + str(runtime) + " seconds")
fig.set_size_inches(18, 6)
plt.show()

Clearly, the equilibrium reached by the system still matches the equilibrium solution. We have however found a set of parameters that can increase available complete AP2 at some point before equilibrium to levels significantly higher than at equilibrium. We don't observe any trapping, but have uncovered an interesting effect. 

Now we'll move on to looking at ARP23. This is 7 subunits, which drastically increases the number of possible reactions. Expect longer runtimes. 

In [ ]:
base_input = '../input_files/arp23.pwr'
rn = ReactionNetwork(base_input, one_step=True)
rn.resolve_tree()
vec_rn = VectorizedRxnNet(rn, dev='cpu')

## Equilibrium Solution for ARP23 ##

Observables and naming are designed to work with single characters, so we use the following scheme:

- ARC2: A
- ARC3: B
- ARC15: C
- ARC18: D
- ARC19: E
- ARC35: F
- ARC40: G

In [ ]:
vec_rn.update_reaction_net(rn)
poly_system = EquilibriumSolver(rn)
solution = poly_system.solve()
print(solution)
print("Equilibrium expected yield: ", 100 * solution[-1] / min(vec_rn.initial_copies[:vec_rn.num_monomers]), '%')

In [ ]:
vec_rn.reset(reset_params=True)
optim = Optimizer(reaction_network=vec_rn,
                  sim_runtime=1,
                  optim_iterations=1000,
                  learning_rate=.01,jk
                  
                  device='cpu')
optim.rn.update_reaction_net(rn)
optim.optimize()

In [ ]:
optim.plot_observable(0)

In [ ]:
optim.plot_observable(-1)

In [ ]:
optim.plot_yield()

In [ ]:
from matplotlib import pyplot as plt
fig, ax = plt.subplots(1, 3)
optim_rn = optim.rn
for i, runtime in enumerate([1, 8, 64]):
    optim_rn.reset()
    sim = VecSim(optim_rn, runtime, device='cpu')
    y = sim.simulate()
    sim.plot_observable(ax=ax[i])
    ax[i].set_title("runtime: " + str(runtime) + " seconds")
fig.set_size_inches(18, 6)
plt.show()